In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import modules
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Import pandas for graphs 
import pandas as pd

# Import the CRUD module
from AnimalShelter import AnimalShelter

# Define username and password

username = "aacuser"
password = "SNHU1234"

# Access database
db = AnimalShelter(username, password)

# Get animal data
df = pd.DataFrame.from_records(db.getAnimal({}))




# Setup the dashboard
app = JupyterDash(__name__)


# Load and display Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Dashboard layout
app.layout = html.Div([
    html.Center(html.B(html.H1('Patrick Kempf Global Rain Dashboard'))),
    html.Img(src=f"data:image/png;base64,{encoded_image}", style={'width': '150px'}),
    html.Hr(),


    # Search features for breed and outcome
    dcc.Input(id='breed-search', type='text', placeholder='Search by Breed'),
    dcc.Input(id='outcome-search', type='text', placeholder='Search by Outcome'),
    
    
    html.Hr(),
    # Filter buttons
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Show All', 'value': 'ALL'},
            {'label': 'Dog', 'value': 'Dog'},
            {'label': 'Cat', 'value': 'Cat'},
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness', 'value': 'Mountain Wilderness'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster Tracking'},
        ],
    value='ALL',  # Get all animals by default
    labelStyle={'display': 'inline-block'}
),
    
    html.Hr(),
    
    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[],
    ),
    html.Br(),
    html.Hr(),
    
    # Graph and map display
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])


# Callback to update table
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value'),
     Input('breed-search', 'value'),
     Input('outcome-search', 'value')]

)
# Functionality for filter buttons and search parameters
def update_dashboard(filter_type, breed_search, outcome_search):

    dff = df
    # Check if there is no data 
    if dff.empty:
        print("DataFrame is empty.")
    # Filters
    if filter_type != 'ALL':
        # Get dogs
        if filter_type == 'Dog':
            dff = dff[dff['animal_type'] == 'Dog']
        # Get cats
        elif filter_type == 'Cat':
            dff = dff[dff['animal_type'] == 'Cat']
        # Get water rescues
        elif filter_type == 'Water Rescue':
            dff = dff[
                (dff['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
                (dff['sex_upon_outcome'] == 'Intact Female') & 
                (dff['age_upon_outcome_in_weeks'].between(26, 156)) &
                (dff['outcome_type'] != 'Euthanasia')
            ]
        # Get mountain/wilderness rescues
        elif filter_type == 'Mountain Wilderness':
            dff = dff[
                (dff['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
                (dff['sex_upon_outcome'] == 'Intact Male') & 
                (dff['age_upon_outcome_in_weeks'].between(26, 156)) &
                (dff['outcome_type'] != 'Euthanasia')
            ]
        # Get disaster and tracking rescues
        elif filter_type == 'Disaster Tracking':
            dff = dff[
                (dff['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
                (dff['sex_upon_outcome'] == 'Intact Male') & 
                (dff['age_upon_outcome_in_weeks'].between(20, 300)) &
                (dff['outcome_type'] != 'Euthanasia')
            ]

    # Search by breed
    if breed_search:
        dff = dff[dff['breed'].str.contains(breed_search, case=False, na=False)]
    # Search by outcome
    if outcome_search:
        dff = dff[dff['outcome_type'].str.contains(outcome_search, case=False, na=False)]
        



    return dff.to_dict('records')

# Callback for graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
# Define breed graph
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if dff.empty:
        return [html.Div("No data to display")]


    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    # Set to bar graph and define parameters
    fig = px.bar(
        breed_counts,
        x='breed',
        y='count',
        title='Preferred Animal Breeds',
        range_y=[0, 100]  
    )
    # Return the graph
    return [
        dcc.Graph(
            figure=fig
        )
    ]

# Callback for map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
# Define the map
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    # Check if index exists 
    if index is None or len(index) == 0:
        return [html.Div("No data available")]

    row = index[0]
    # Return map with defined parameters
    return [
        dl.Map(style={'width': '500px', 'height': '500px'},
               center=[dff.iloc[row, 13], dff.iloc[row, 14]], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row, 4]), 
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row, 9]) 
                                 ])
                             ])
               ])
    ]
# Run server
app.run_server(debug=True)


Dash app running on http://127.0.0.1:26479/
